<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C3/W3/ungraded_labs/C3_W3_Lab_2_multiple_layer_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2022-11-26 19:38:03--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.128, 74.125.68.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  23.6MB/s    in 4.6s    

2022-11-26 19:38:08 (18.4 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [2]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [3]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [4]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [5]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [6]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [7]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [8]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'"The Incredible Melting Man" is a fantastically gross, trashy and energetic Z-grade production that every self-respecting camp-horror freak simply has to see for him/herself! The ideal way to describe this low-budget 70\'s gem is like a shameless copy of Hammer\'s "The Quatermass Xperiment" ...only a thousand times filthier! Astronaust Steve West is the only survivor of a disastrous space-mission, but turns out the carrier of a horrible disease that makes him radioactive and ... causes him to melt! In shock after seeing his face in the mirror (can you blame him?), Steve busts out of the hospital, leaving a trail of sticky pus and fallen off body parts behind. Doctor Ted Nelson has to find him urgently, as the disease also set Steve up with an insatiable appetite for human flesh. The premise may sound utterly stupid but this flick is enormously entertaining and contains great make-up effects from the hand of Rick Baker. The melting dude\'s face looks like a rotting pizza and 

In [9]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [10]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [11]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  2,   1,   1, ...,   0,   0,   0],
       [ 10,   1, 532, ...,   0,   0,   0],
       [  1, 257,   8, ...,  13,  13,  13]])

In [12]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'"The Incredible Melting Man" is a fantastically gross, trashy and energetic Z-grade production that every self-respecting camp-horror freak simply has to see for him/herself! The ideal way to describe this low-budget 70\'s gem is like a shameless copy of Hammer\'s "The Quatermass Xperiment" ...only a thousand times filthier! Astronaust Steve West is the only survivor of a disastrous space-mission, but turns out the carrier of a horrible disease that makes him radioactive and ... causes him to melt! In shock after seeing his face in the mirror (can you blame him?), Steve busts out of the hospital, leaving a trail of sticky pus and fallen off body parts behind. Doctor Ted Nelson has to find him urgently, as the disease also set Steve up with an insatiable appetite for human flesh. The premise may sound utterly stupid but this flick is enormously entertaining and contains great make-up effects from the hand of Rick Baker. The melting dude\'s face looks like a rotting pizza an

In [13]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [14]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True, True]


In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
loss=0.1
accuracy = 0.88
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss') <= loss): # Experiment with changing this value
      print("\nReached 85% accuracy so cancelling training!")
      self.model.stop_training = True
      
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=20, callbacks=myCallback())

Epoch 1/10
391/391 [==============================] - 47s 89ms/step - loss: 0.4635 - accuracy: 0.7736 - val_loss: 0.3884 - val_accuracy: 0.8258
Epoch 2/10
391/391 [==============================] - 30s 77ms/step - loss: 0.3465 - accuracy: 0.8525 - val_loss: 0.3801 - val_accuracy: 0.8281
Epoch 3/10
391/391 [==============================] - 30s 77ms/step - loss: 0.3201 - accuracy: 0.8645 - val_loss: 0.3460 - val_accuracy: 0.8539
Epoch 4/10
391/391 [==============================] - 30s 77ms/step - loss: 0.3037 - accuracy: 0.8720 - val_loss: 0.3364 - val_accuracy: 0.8516


KeyboardInterrupt: ignored

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [19]:
# predict on a sample text without padding.

sample_text = input("Type in review to analyze here: ")
predictions = model.predict(np.array([sample_text]))

if predictions[0]<0.5:
  percentage = np.round((1-predictions.item(0))*100,1)
  sentiment = "<NEGATIVE REVIEW>"
  print(percentage,'% chance of:',sentiment)
else: 
  percentage = np.round(predictions.item(0)*100,1)
  sentiment = "<POSITIVE REVIEW>"
  print(percentage,'% chance of:',sentiment)


Type in review to analyze here: I just want to put my point of view out there it is no longer sarducci's it's Genoa's Pizza today was my first time ordering I see a lot of bad reviews and as far as I'm concerned for a place that's open at night in New Bedford it was really good food I only got a cheese pizza and a BLT but I'll tell you it was delicious please stop mistaken this place for sarducci's I don't know the owner I don't know anybody that's there but I will tell you give this place a try
1/1 [==============================] - 0s 25ms/step
60.7 % chance of: <POSITIVE REVIEW>


In [ ]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])